In [ ]:
import random
import math
from collections import Counter
from nltk.corpus import europarl_raw
from nltk import ngrams
from sklearn.model_selection import train_test_split
from nltk.corpus import words

# The nltk version is 3.2.2.
# The scikit-learn version is 0.18.1.
#corpus size 19899 sentences
#alpha value is chosen using the validation set
alpha = 0.023 
corpus = europarl_raw.english
# Delta estimation based on Good Turing
delta = 0.75
P_cont_denom_bigrams = 0

#split(sentences) into train-->60% validation-->20% and test-->20%
train_init, test = train_test_split(corpus.sents(), train_size = 0.8,random_state=4542)
train, validation = train_test_split(train_init, train_size = 0.75,random_state=4572)

def getCount(countdict,key):

    '''Helper function that returns # of uni/bi/tri -grams(key) occurencies in corpus.
    If there are not such occurencies in corpus returns 0
    '''
    try:
        return countdict[key]
    except KeyError:
        return 0  
    
cnt = Counter()
for word in [item for sublist in train for item in sublist]:
    cnt[word] += 1

#Remove rare words 
unigrams = { k:v for k, v in cnt.items() if v >= 10}
unigrams['<s>'] = 2*len(train) #every sentence starts and ends with <s> symbol
unigrams_size = sum(v for v in unigrams.values())
V = len(unigrams)


bigrams =  (ngram for sent in train for ngram in ngrams(sent, 2,
    pad_left=True, pad_right=True, left_pad_symbol='<s>',right_pad_symbol='<s>'))

cnt2 = Counter()
for bigram in bigrams:
    cnt2[bigram] += 1


    
#remove bigrams containing rare words
bigrams_final = { k:v for k, v in cnt2.items() if ((k[0] in unigrams.keys()) and(k[1] in unigrams.keys())) }
bigrams_size = sum(v for v in bigrams_final.values())

#Auxiliary data structure for bigrams that  used in order to improve Knesser-Ney performance
bigrams_KN ={}
for k, v in bigrams_final.items():
    bigrams_KN[k[0]] = {}
for k, v in bigrams_final.items():
    bigrams_KN[k[0]][k[1]] = v

#Auxiliary data structure for bigrams that used in order to improve Knesser-Ney performance  
bigrams_KN_reverse ={}
for k, v in bigrams_final.items():
    bigrams_KN_reverse[k[1]] = {}
for k, v in bigrams_final.items():
    bigrams_KN_reverse[k[1]][k[0]] = v

#compute P(continuation) denominator for bigrams
wordsNum = []
for k,v in bigrams_final.items():
    wordsNum.append(k[0])
P_cont_denom_bigrams = len(set(wordsNum))
trigrams = (ngram for sent in train for ngram in ngrams(sent, 3,
            pad_left=True, pad_right=True, left_pad_symbol='<s>',right_pad_symbol='<s>'))


cnt3 = Counter()
for trigram in trigrams:
    cnt3[trigram] += 1

#remove trigrams containing rare words
trigrams_final = { k:v for k, v in cnt3.items() if ((k[0] in unigrams.keys())
    and(k[1] in unigrams.keys()) and(k[2] in unigrams.keys())) }
trigrams_size = sum(v for v in trigrams_final.values())




def unigram_logprob_ls(unigram):

    # laplace smoothing
    return math.log2(((getCount(unigrams,unigram))+alpha) / (unigrams_size+(alpha*V)) )

def bigram_logprob_ls(bigram):

    # laplace smoothing
    return math.log2(((getCount(bigrams_final,bigram))+alpha ) / ((getCount(unigrams,(bigram[0])))+(alpha*V))) 
    
def bigram_logprob_Mod_KN(bigram):

    # Modified Knesser ney 
    try:
        return math.log2(((max((getCount(bigrams_final,bigram) -delta),0)) / getCount(unigrams,(bigram[0]))) 
                     +((bigram_Mod_KN_lamda(bigram[0])) * bigram_Mod_KN_Pcont(bigram[1])))
    
    except (ZeroDivisionError,ValueError):
        '''return bigram prob with laplace smoothing if both bigram does not exist
        and P(continuation) or/and lamda is  zero
        '''
        return bigram_logprob_ls(bigram)
    
    
def bigram_Mod_KN_lamda(word):

    '''Compute Modified Knesser ney  interpolation term(lamda)
    Compute The number of word types that can follow word
    '''
    try:
        times = len(bigrams_KN[word])
    except KeyError:
        times =  0
    lamda = (delta/getCount(unigrams,word)) * times
    return lamda

def bigram_Mod_KN_Pcont(word):

    '''Compute P continuation
    Count distinct vocabulary words seen to proceede word
    '''
    try:
        count = len(bigrams_KN_reverse[word])
    except KeyError:
        count = 0
    return count/P_cont_denom_bigrams
    

def trigram_logprob_ls(trigram):

    # laplace smoothing
    return math.log2(((getCount(trigrams_final,trigram)) +alpha ) /
                     ((getCount(bigrams_final,(trigram[0],trigram[1])))+(alpha*V)))   

def logprob_sentence_bigram(sentence):
    
    sumprob = 0
    for i in range(len(sentence)-1):
        sumprob += bigram_logprob_ls((sentence[i],sentence[i+1]))
    return sumprob

def logprob_sentence_bigram_Mod_KN(sentence):
    
    sumprob = 0
    for i in range(len(sentence)-1):
        sumprob += bigram_logprob_Mod_KN((sentence[i],sentence[i+1]))
    return sumprob       
    
def logprob_sentence_trigram(sentence):
    
    sumprob = 0
    for i in range(len(sentence)-2):
        sumprob += trigram_logprob_ls((sentence[i],sentence[i+1],sentence[i+2]))
    return sumprob


    
def bigram_model_perplexity(corpus):
    
    #Compute cross entropy and perplexity of our bigram model
    sumprob = 0
    bigram_count = 0
    for sentence in corpus:
        sentence = ['<s>'] + sentence + ['<s>'] 
        bigram_count += (len(sentence) -1)
        sumprob += logprob_sentence_bigram(sentence)
    cross_entropy = -sumprob/bigram_count
    perpl = math.pow(2,cross_entropy)
    return cross_entropy,perpl


def bigram_model_perplexity_Mod_KN(corpus):
    
    #Compute cross entropy and perplexity of our bigram model with Modified Knesser-Ney Smoothing
    sumprob = 0
    bigram_count = 0
    for sentence in corpus:
        sentence = ['<s>'] + sentence + ['<s>'] 
        bigram_count += (len(sentence) -1)
        sumprob += logprob_sentence_bigram_Mod_KN(sentence)
    cross_entropy = -sumprob/bigram_count
    perpl = math.pow(2,cross_entropy)
    return cross_entropy,perpl

def trigram_model_perplexity(corpus):
    
    #Compute cross entropy and perplexity of our trigram model
    sumprob = 0
    trigram_count = 0
    for sentence in corpus:
        sentence = ['<s>','<s>'] + sentence + ['<s>','<s>'] 
        trigram_count += (len(sentence) -2)
        sumprob += logprob_sentence_trigram(sentence)
    cross_entropy = -sumprob/trigram_count
    perpl = math.pow(2,cross_entropy)
    return cross_entropy,perpl
    
def score_sentence_vs_random(corpus):
    
    sent= random.choice(corpus)
    sent = ['<s>','<s>'] + sent + ['<s>','<s>'] 
    score_sent = logprob_sentence_trigram(sent)
    print(sent,"score:",score_sent)
    random_sent = []
    for i in range(len(sent)-4):
        random_sent.append(random.choice(words.words()))
    random_sent =['<s>','<s>']+ random_sent + ['<s>','<s>']
    score_random_sent = logprob_sentence_trigram(random_sent)
    print(random_sent,"score:",score_random_sent)

def predict_next_word(sentence):
    
    #predict next word based on most frequent relevant trigrams/ bigrams
    suggestions = []
    if (len(sentence) >= 2):
        trigrams = { k:v for k, v in trigrams_final.items() if ((k[0] == sentence[-2])
            and(k[1] == sentence[-1]))  }
        if (len(trigrams)>0):
            sortdict = [(k, trigrams[k]) for k in sorted(trigrams, key=trigrams.get, reverse=True)]
            for k,v in sortdict[:3]:
                suggestions.append(k[2])
            
    else:
        bigrams = { k:v for k, v in bigrams_final.items() if ((k[0] == sentence[-1])) }
        if (len(bigrams)>0):
            sortdict = [(k, bigrams[k]) for k in sorted(bigrams, key=bigrams.get, reverse=True)]
            for k,v in sortdict[:3]:
                suggestions.append(k[1])
    
    return suggestions
       
def bigram_trigram_interpolation_prob(trigram,lamda1,lamda2):
    
    #interpolate bigrams and trigrams
    assert(lamda1+lamda2==1.0),"error interpolation coefs must sum to 1!"
    return ((lamda1*trigram_logprob_ls((trigram[0],trigram[1],trigram[2]))) + 
             (lamda2*bigram_logprob_ls((trigram[1],trigram[2]))))

def interpolated_sentence(sentence,lamda1,lamda2):
    
    #propability of sentence using the interpolation of the bigram/trigram models
    sumprob = 0
    for i in range(len(sentence)-2):
        sumprob += bigram_trigram_interpolation_prob((sentence[i],sentence[i+1],sentence[i+2]),lamda1,lamda2)
    return sumprob

def interpolated_model_perplexity(corpus,lamda1,lamda2):
    
    #Compute cross entropy and perplexity of our trigram model
    sumprob = 0
    trigram_count = 0
    for sentence in corpus:
        sentence = ['<s>','<s>'] + sentence + ['<s>','<s>'] 
        trigram_count += (len(sentence) -2)
        sumprob += interpolated_sentence(sentence,lamda1,lamda2)
    cross_entropy = -sumprob/trigram_count
    perpl = math.pow(2,cross_entropy)
    return perpl

#Use validation data to tune alpha parameter
# for i in range(1,101,1):
#     alpha = i/1000
#     print("alpha:", alpha)
#     print(bigram_model_perplexity(validation))

#     print(trigram_model_perplexity(validation))


# for i in range(0, 101, 1):
#     lamda1 = i/100
#     lamda2 = 1-lamda1
#     print("lamda1: ",lamda1," lamda2: ",lamda2," perplexity--> "
#           ,interpolated_model_perplexity(validation,lamda1,lamda2))
#Best model where lamda1=0 and lamda2 =1
    


print(bigram_model_perplexity_Mod_KN(train))
print(bigram_model_perplexity(train))
print(trigram_model_perplexity(train))
print(bigram_model_perplexity_Mod_KN(test))
print(bigram_model_perplexity(test))
print(trigram_model_perplexity(test))

score_sentence_vs_random(test)
print(predict_next_word(["treaty"]))

#Results @ TRAIN(Cross Entropy, perplexity)
# (6.069444114679039, 67.15598686977955)--> bigram Knesser-Kney 
# (6.833099295053231, 114.01653831970505)--> bigram laplace smoothing 
# (6.198075421538283, 73.41868764893984)--> trigram laplace smoothing

#Results @ TEST(Cross Entropy, perplexity)
# (6.339900707297582, 81.00284684531835)--> bigram Knesser-Kney 
# (7.666218268695851, 203.1241926649754)--> bigram laplace smoothing
# (8.34177441486956, 324.4324722136551) --> trigram laplace smoothing





In [ ]:
import numpy as np
from pqdict import pqdict
CLOSEST_WORDS = 5
VOCABULARY = unigrams.keys()


def DP_table(word1,word2):
    
    ''' Construct DP table,in order to compute Levenstein distance
     Use a list of lists as data structure for DP table
    '''
    dist_table = []
    word2_list = list(word2)
    for i in range(len(word2)+2):
        table_row = []
        if (i == 0):
            table_row.append("")
            table_row.append("#")
            table_row.extend(list(word1))
        elif (i == 1):
            table_row.append("#")
            table_row.extend(range(len(word1)+1))
        else:
            table_row.append(word2_list[i-2])
            table_row.append(i-1)
            table_row.extend([0]*len(word1))
        dist_table.append(table_row)
   
    return(dist_table)


def compute_distance(word1,word2):
    
    '''fill row by row the DP table with the calculated Levenstein distance
    according to the associated cost below
    '''
    insertion = 1
    deletion =1
    replace = 2
    
    #Construct & initialize DP table
    dp_table = DP_table(word1,word2)
    for i in range(2,(len(word2)+2)):
        for j in range(2,(len(word1)+2)):
            if dp_table[0][j] ==  dp_table[i][0]:
                dp_table[i][j] = min((dp_table[i][j-1]+insertion),(dp_table[i-1][j]+deletion),(dp_table[i-1][j-1]))
            else:
                dp_table[i][j] = min((dp_table[i][j-1]+insertion),(dp_table[i-1][j]+deletion),(dp_table[i-1][j-1]+replace))
            
   
    return dp_table
    




def comp_closest_words(word):
    
    '''Compute a number(CLOSEST_WORDS global variable) of closest words(based on Levenstein distance)
    of the parsed word. The set of unigrams is used as vocabulary 
    
    '''
    pq = pqdict()
    for voc_word in VOCABULARY:
        dist = compute_distance(word,voc_word)
        pq.update({voc_word:dist[-1][-1]})
    closest_words = {}       
    for idx in range(CLOSEST_WORDS):
        closest_words.update({pq.popitem()})
    return closest_words




def viterbi_corrector(sentence):
    
    '''
    Implements a viterbi decoder for context-sensitive spelling corrector. The recursive formula of the
    viterbi decoder is: V(k) = LD(k) * max(P(wk|wk-1)* Vk-1(wk-1)) , where LD is the Levestein distance
    from the sentence's words and P(wk|wk-1) the log propability of bigram(wk,wk-1)
    '''
    
    Words = np.empty([CLOSEST_WORDS, len(sentence)],dtype = 'U10000')
    LD_Distance = np.empty([CLOSEST_WORDS, len(sentence)],dtype = int)

    for idx in range(len(sentence)):
        closest_words = comp_closest_words(sentence[idx])
        jdx = 0
        for k,v in closest_words.items():
            Words[jdx,idx] = k
            LD_Distance[jdx,idx] = v
            jdx +=1

    Vit = np.empty([CLOSEST_WORDS, len(sentence)],dtype = 'f')
    Vit_trace = np.empty([CLOSEST_WORDS, len(sentence)-1],dtype = int)

    for jdx in range(len(sentence)):
        for idx in range(CLOSEST_WORDS):
                if (jdx ==0):
                    #Added +1 to LD distance prevents original sentence's words(LD=0) to always be chosen as max possible
                    Vit[idx,jdx]= bigram_logprob_Mod_KN(("<s>",Words[idx,jdx])) * (LD_Distance[idx,jdx]+1)
                else:
                    bigram_logprob_Mod_KN(("<s>",Words[idx,jdx]))
                    maxprob = -float("inf")
                    trace = 0
                    for idx2 in range(CLOSEST_WORDS):
                        prob = Vit[idx2,jdx-1] +(bigram_logprob_Mod_KN((Words[idx2,jdx-1],Words[idx,jdx]))
                                                 * (LD_Distance[idx,jdx]+1))
                        if (prob > maxprob):
                            maxprob = prob
                            trace = idx2
                    Vit[idx,jdx] = maxprob
                    Vit_trace[idx,jdx-1] = trace

    max_endprob = -float("inf")
    end_trace = 0
    for idx in range(CLOSEST_WORDS):                        
        prob = Vit[idx,-1] +(bigram_logprob_Mod_KN((Words[idx,-1],"<s>"))  * (LD_Distance[idx,-1]+1))
        if (prob > maxprob):
            max_endprob = prob
            end_trace = idx




    words_idx = []
    words_idx.append(end_trace)
    idx = len(sentence)-2
    while (idx >= 0):
        words_idx.append(Vit_trace[words_idx[-1],idx])
        idx -=1

    words_idx = list(reversed(words_idx))


    corrected_sentence = []
    for idx in range(len(sentence)):
        corrected_sentence.append(Words[words_idx[idx],idx])
    return corrected_sentence
    



def greedy_corrector(sentence):
    
    '''
    Implements a baseline context-sensitive spelling corrector. The spelling corrector
    is going to take into account only the max P(wk|wk-1) and the Levestein distance
    from the sentence's words(greedy choice). The formula is  P(k) = LD(k) * max(P(wk|wk-1)
    
    '''
    
    Words = np.empty([CLOSEST_WORDS, len(sentence)],dtype = 'U10000')
    LD_Distance = np.empty([CLOSEST_WORDS, len(sentence)],dtype = int)

    for idx in range(len(sentence)):
        closest_words = comp_closest_words(sentence[idx])
        jdx = 0
        for k,v in closest_words.items():
            Words[jdx,idx] = k
            LD_Distance[jdx,idx] = v
            jdx +=1

    corrected_sentence = []
    maxprob = -float("inf")
    index = 0
    for idx in range(CLOSEST_WORDS):
        prob = bigram_logprob_Mod_KN(("<s>",Words[idx,0])) * (LD_Distance[idx,0]+1)
        if prob > maxprob:
            prob = maxprob
            index = idx
    corrected_sentence.append(Words[index,0])
    
    for jdx in range(1,len(sentence)):
        maxprob = -float("inf")
        index = 0
        for idx in range(CLOSEST_WORDS):
            prob = bigram_logprob_Mod_KN((corrected_sentence[-1],Words[idx,jdx])) * (LD_Distance[idx,jdx]+1)
            if prob > maxprob:
                prob = maxprob
                index = idx
        corrected_sentence.append(Words[index,jdx])
        
    
    return corrected_sentence
    
    



In [ ]:
import string
import random

TEST_SIZE = 500

def perword_accuracy(sentence,corr_sentence):
    
    #calculate per-word accuracy of the corrected sentence
    assert(len(sentence) == len(corr_sentence)),"error the length of correct and corrected sentence must be equal"
    acc = 0
    for idx in range(len(sentence)):
        if sentence[idx] == corr_sentence[idx]:
            acc +=1
            
    return acc/len(sentence)
    
    
  
    
    
    
def insert_errors(sentence):
    
    #Indroduce random spelling errors to a given sentence
    letters = list(string.ascii_lowercase)
    new_sentence = []
    for idx in range(len(sentence)):
        # add spelling error with 0.5 prob for each word
        dice = random.randint(1,2)
        if (dice == 1):
            word = list(sentence[idx])
            if (len(word) < 5):
                word.append( random.choice(letters))
            else:
                idx1 = random.randint(0,len(word)-1)
                idx2 = random.randint(0,len(word)-1)
                word[idx1] = random.choice(letters)
                word[idx2] = random.choice(letters)
            word = "".join(word)
            new_sentence.append(word)
            
        else:
             new_sentence.append(sentence[idx])
       
    return new_sentence
              

#Select randomly a sample(of size TEST_SIZE) from test data
test_sample =random.sample(test, TEST_SIZE)

per_word_acc_viterbi = 0
per_word_acc_greedy = 0
per_word_acc_baseline = 0

for sent in test_sample:
    n_sen = insert_errors(sent)
    per_word_acc_viterbi+= perword_accuracy(sent,viterbi_corrector(n_sen))
    per_word_acc_greedy+= perword_accuracy(sent,greedy_corrector(n_sen))
    per_word_acc_baseline+= perword_accuracy(sent,n_sen)
    
avg_per_word_acc_viterbi = per_word_acc_viterbi/TEST_SIZE
avg_per_word_acc_greedy = per_word_acc_greedy/TEST_SIZE
avg_per_word_acc_baseline = per_word_acc_baseline/TEST_SIZE

print("Viterbi average per word accuracy: ", avg_per_word_acc_viterbi)
print("Greedy average per word accuracy: ", avg_per_word_acc_greedy)
print("Baseline average per word accuracy: ", avg_per_word_acc_baseline)


# Viterbi average per word accuracy:  0.8212886189248795
# Greedy average per word accuracy:  0.14442630786640534
# Baseline average per word accuracy:  0.5021541453981966